# Project 03 - Due Monday, November 19 at 1pm

*Objectives*: Use Spark to process and perform basic analysis on non-relational data, including its DataFrame and SQL interfaces.

*Grading criteria*: The tasks should all be completed, and questions should all be answered with Python code, SQL queries, shell commands, and markdown cells.  The notebook itself should be completely reproducible (using AWS EC2 instance based on the provided AMI) from start to finish; another person should be able to use the code to obtain the same results as yours.  Note that you will receive no more than partial credit if you do not add text/markdown cells explaining your thinking when appropriate.

*Attestation*: **Work in groups**.  At the end of your submitted notebook, identify the work each partner performed and attest that each contributed substantially to the work.

*Deadline*: Monday, November 19, 1pm.  One member of each group must submit your notebook to Blackboard; you should not submit it separately.

## Part 1 - Setup

Begin by setting up Spark and fetching the project data.  

**Note**: you may want to use a larger EC2 instance type than normal.  This project was prepared using a `t2.xlarge` instance.  Just remember that the larger the instance, the higher the per-hour charge, so be sure to remember to shut your instance down when you're done, as always.

### About the data

We will use JSON data from Twitter; we saw an example of this in class.  It should parse cleanly, allowing you to focus on analysis.

This data was gathered using GWU Libraries' [Social Feed Manager](http://sfm.library.gwu.edu/) application during a recent game of the MLB World Series featuring the Los Angeles Dodgers and Boston Red Sox.  This first file tells you a little bit about how it was gathered:

In [1]:
!wget https://s3.amazonaws.com/2018-dmfa/project-3/ea26ccd641744d4a8dce84de0785186d-README.txt

--2018-11-19 03:56:59--  https://s3.amazonaws.com/2018-dmfa/project-3/ea26ccd641744d4a8dce84de0785186d-README.txt
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.136.182
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.136.182|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1145 (1.1K) [text/plain]
Saving to: ‘ea26ccd641744d4a8dce84de0785186d-README.txt.1’

ea26ccd641744d4a8dc 100%[===================>]   1.12K  --.-KB/s    in 0s      

2018-11-19 03:56:59 (97.8 MB/s) - ‘ea26ccd641744d4a8dce84de0785186d-README.txt.1’ saved [1145/1145]



In [2]:
!cat ea26ccd641744d4a8dce84de0785186d-README.txt

This is an export created with Social Feed Manager.

EXPORT INFORMATION
Selected seeds: All seeds
Export id: ea26ccd641744d4a8dce84de0785186d
Export type: twitter_filter
Format: Full JSON
Export completed:  Oct. 30, 2018, 9:45:59 a.m. EDT
Deduplicate: Yes

COLLECTION INFORMATION
Collection name: 2018-world-series

Collection id: 4e2564448b144915b6a0eb1899075a44
Collection set: 2018-mlb-playoffs (collection set id 5a00efa0bddf4be2aa19c6df9788ff6e)
Harvest type: Twitter filter

Harvest options:

Seeds:
* Track: dodgers,redsox,red sox,bossox,world series,RedSoxVsDodgers,ladodgers,bostonredsox - Active

Change log:

Change to 2018-world-series (collection) on Oct. 30, 2018, 8:16:14 a.m. EDT by dchud:
Note: Series ended Sunday night after five games, waited ~36 hours.

Change to 2018-world-series (collection) on Oct. 23, 2018, 6:01:28 p.m. EDT by dchud:

Change to Track: dodgers,redsox,red sox,bossox,world series,RedSoxVsDodgers,ladodgers,bostonredsox (seed) on Oct. 23, 2018, 6:01:18 p.m. E

The most important pieces in that metadata are:

 * It tracked tweets that mentioned "dodgers" or "redsox" and several additional related terms.  Every item in this set should refer to one or more of these terms.
 * This data was deduplicated; we should not see individual tweets more than once.
 * Data was collected between October 23 and October 30.  All five games of the Series were played during this time.
 
You should not need to know anything about baseball to complete this assignment, but if you have baseball questions (or Twitter questions!) please ask on the discussion forum.

**Please note**: sometimes social media data contains offensive material.  This data set has not been filtered; if you do come across something inappropriate, please do your best to ignore it if you can.

## Fetch the data

The following files are available:

 * https://s3.amazonaws.com/2018-dmfa/project-3/ea26ccd641744d4a8dce84de0785186d_009.json
 * https://s3.amazonaws.com/2018-dmfa/project-3/ea26ccd641744d4a8dce84de0785186d_010.json
 * https://s3.amazonaws.com/2018-dmfa/project-3/ea26ccd641744d4a8dce84de0785186d_011.json
 * https://s3.amazonaws.com/2018-dmfa/project-3/ea26ccd641744d4a8dce84de0785186d_012.json
 
### Q1.1 - Select at least one and obtain it using `wget`.  Verify the file sizes using the command line.

Each file should contain exactly 100,000 tweets.  

*Note*: you are only required to use one of these files, but you may use more than one.  It will be easier to process more data if you use a larger EC2 instance type, as suggested above.  Use the exact same set of files throughout the assignment.

**Answer**

In [3]:
!wget https://s3.amazonaws.com/2018-dmfa/project-3/ea26ccd641744d4a8dce84de0785186d_009.json

--2018-11-19 03:57:00--  https://s3.amazonaws.com/2018-dmfa/project-3/ea26ccd641744d4a8dce84de0785186d_009.json
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.136.182
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.136.182|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 554277526 (529M) [application/json]
Saving to: ‘ea26ccd641744d4a8dce84de0785186d_009.json.1’

ea26ccd641744d4a8dc 100%[===================>] 528.60M  51.2MB/s    in 9.6s    

2018-11-19 03:57:09 (55.2 MB/s) - ‘ea26ccd641744d4a8dce84de0785186d_009.json.1’ saved [554277526/554277526]



The file do have 100000 tweets.

In [4]:
!wc -l ea26ccd641744d4a8dce84de0785186d_009.json

100000 ea26ccd641744d4a8dce84de0785186d_009.json


For your reference, here is the text of one Tweet, randomly selected from one of these files.  You might wish to study its structure and refer to it later.

In [5]:
!cat *.json | shuf -n 1 > example-tweet.json

In [6]:
import json
print(json.dumps(json.load(open("example-tweet.json")), indent=2))

{
  "entities": {
    "symbols": [],
    "media": [
      {
        "source_status_id": 1056742515749994496,
        "source_status_id_str": "1056742515749994496",
        "media_url": "http://pbs.twimg.com/media/DqpME3TXgAA1bIB.jpg",
        "display_url": "pic.twitter.com/YHul61YyDD",
        "expanded_url": "https://twitter.com/RedSox/status/1056742515749994496/photo/1",
        "media_url_https": "https://pbs.twimg.com/media/DqpME3TXgAA1bIB.jpg",
        "source_user_id": 40918816,
        "id": 1056740709020172288,
        "sizes": {
          "large": {
            "resize": "fit",
            "h": 576,
            "w": 1024
          },
          "medium": {
            "resize": "fit",
            "h": 576,
            "w": 1024
          },
          "small": {
            "resize": "fit",
            "h": 383,
            "w": 680
          },
          "thumb": {
            "resize": "crop",
            "h": 150,
            "w": 150
          }
        },
        "indices"

You can find several key elements in this example; the text, time, and language of the tweet, whether it was a reply to another user, the user's screen name along with their primary language and other account information like creation date, follower/friend/tweet counts, and perhaps their location.  If there are hashtags, user mentions, or urls present in their tweet, they will be present in the `entities` section; these are not present in every tweet.  If this is a retweet, you will see the original tweet and its information nested within.

### Q1.2 - Start up Spark, and verify the file sizes.

We will use our normal startup sequence here:

In [7]:
import findspark

In [8]:
findspark.init()

In [9]:
from pyspark import SparkContext

In [10]:
spark = SparkContext(appName='project-03')

In [11]:
spark

<SparkContext master=local[*] appName=project-03>

In [12]:
from pyspark import SQLContext

In [13]:
sqlc = SQLContext(spark)

In [14]:
sqlc

In [15]:
tweets = sqlc.read.json("ea26ccd6*.json")

Verify that Spark has loaded the same number of tweets you saw before:

**Answer**

10000, same as it was counted by using _! wc_ command

In [16]:
tweets.count()

100000

## Part 2 - Comparing DataFrames and Spark SQL

For the next three questions, we will look at operations using both DataFrames and SQL queries. Note that `tweets` is already a DataFrame:

In [17]:
tweets

DataFrame[contributors: string, coordinates: struct<coordinates:array<double>,type:string>, created_at: string, display_text_range: array<bigint>, entities: struct<hashtags:array<struct<indices:array<bigint>,text:string>>,media:array<struct<additional_media_info:struct<description:string,embeddable:boolean,monetizable:boolean,title:string>,display_url:string,expanded_url:string,id:bigint,id_str:string,indices:array<bigint>,media_url:string,media_url_https:string,sizes:struct<large:struct<h:bigint,resize:string,w:bigint>,medium:struct<h:bigint,resize:string,w:bigint>,small:struct<h:bigint,resize:string,w:bigint>,thumb:struct<h:bigint,resize:string,w:bigint>>,source_status_id:bigint,source_status_id_str:string,source_user_id:bigint,source_user_id_str:string,type:string,url:string>>,symbols:array<struct<indices:array<bigint>,text:string>>,urls:array<struct<display_url:string,expanded_url:string,indices:array<bigint>,url:string>>,user_mentions:array<struct<id:bigint,id_str:string,indices:a

To issue SQL queries, we need to register a table based on `tweets`:

In [18]:
tweets.createOrReplaceTempView("tweets")

### Q2.1 - Which 10 languages are most commonly used in tweets?  Verify your result by executing it with both the dataframe and with SQL.

Hint: for the dataframe, use `groupBy`, `count`, and `orderBy`.  See the documentation at https://spark.apache.org/docs/2.3.1/api/python/pyspark.sql.html for details on these and other functions.

**Answer**

We have a look at the name of the columns

In [20]:
tweets.columns

['contributors',
 'coordinates',
 'created_at',
 'display_text_range',
 'entities',
 'extended_entities',
 'extended_tweet',
 'favorite_count',
 'favorited',
 'filter_level',
 'geo',
 'id',
 'id_str',
 'in_reply_to_screen_name',
 'in_reply_to_status_id',
 'in_reply_to_status_id_str',
 'in_reply_to_user_id',
 'in_reply_to_user_id_str',
 'is_quote_status',
 'lang',
 'place',
 'possibly_sensitive',
 'quote_count',
 'quoted_status',
 'quoted_status_id',
 'quoted_status_id_str',
 'quoted_status_permalink',
 'reply_count',
 'retweet_count',
 'retweeted',
 'retweeted_status',
 'source',
 'text',
 'timestamp_ms',
 'truncated',
 'user']

__Use Dataframe__

__Use SQL__

In [21]:
tweets.select('lang').take(1)

[Row(lang='en')]

The GROUP BY statement is  used with aggregate functions to group the result-set by language. We count the number, decent the count and show the top ten, verify the answer by SQL

In [22]:
group = tweets.groupBy('lang').count()
from pyspark.sql.functions import *
group.sort(desc('count')).show(10)

+----+-----+
|lang|count|
+----+-----+
|  en|88545|
| und| 5217|
|  es| 4927|
|  pt|  402|
|  fr|  266|
|  ht|  136|
|  ja|   98|
|  tl|   86|
|  in|   55|
|  ca|   41|
+----+-----+
only showing top 10 rows



In [23]:
sqlc.sql("""
    SELECT lang, COUNT(lang)
    FROM tweets
    GROUP BY lang
    ORDER BY COUNT(lang) DESC
""").show(10)

+----+-----------+
|lang|count(lang)|
+----+-----------+
|  en|      88545|
| und|       5217|
|  es|       4927|
|  pt|        402|
|  fr|        266|
|  ht|        136|
|  ja|         98|
|  tl|         86|
|  in|         55|
|  ca|         41|
+----+-----------+
only showing top 10 rows



### Q2.2 - Which 10 time zones are most common among users?  Verify your result with both the dataframe and SQL.

*Note*: for this question, you may leave NULL values present in your results, as a way to help you understand what data is present and what is missing.

**Answer**

In [24]:
group_user = tweets.groupBy('user.time_zone').count()
group_user.sort(desc('count')).show(10)

+---------+------+
|time_zone| count|
+---------+------+
|     null|100000|
+---------+------+



In [25]:
sqlc.sql("""
    SELECT user.time_zone
    FROM tweets
    GROUP BY user.time_zone
    ORDER BY COUNT(user.time_zone) DESC
""").show(10)

+---------+
|time_zone|
+---------+
|     null|
+---------+



The file we selected has null values for 'time_zone'

### Q2.3 - How many tweets mention the Dodgers?  How many mention the Red Sox?  How many mention both?

You may use either the dataframe or SQL to answer.  Explain why you have chosen that approach.

Hint:  you will want to look at the value of the `text` field.

**Answer**

The LIKE operator is used in a WHERE clause to search for a specified pattern in a column.  The percent sign represents zero, one, or multiple characters

__Count the number of tweets that mentioned Dodgers__

In [26]:
sqlc.sql("""
    SELECT COUNT(text) AS COUNT 
    FROM tweets
    WHERE text LIKE '%Dodgers%'
""").show()

+-----+
|COUNT|
+-----+
|20745|
+-----+



__Count the number of tweets that mentioned Red Sox__

In [27]:
sqlc.sql("""
    SELECT COUNT(text) AS COUNT 
    FROM tweets
    WHERE text LIKE '%Red Sox%'
""").show()

+-----+
|COUNT|
+-----+
|14863|
+-----+



__Count the number of tweets that mentioned Red Sox and Dodgers__

In [28]:
sqlc.sql("""
    SELECT COUNT(text) AS COUNT 
    FROM tweets
    WHERE text LIKE '%Red Sox%'
    AND text LIKE '%Dodgers%'
""").show()

+-----+
|COUNT|
+-----+
| 2625|
+-----+



## Part 3 - More complex queries

For this section, you may choose to use dataframe queries or SQL.  If you wish, you may verify results by using both, as in Part 2, but this is not required for this section.

### Q3.1 - Team mentions by location

In which users' locations are the Red Sox and the Dodgers being mentioned the most?  Consider each team separately, one at a time.  Discuss your findings.

Hint:  you may use either the time zones or user-specified locations for this question.

**Answer**

We count the user-specified locations.The ORDER BY keyword is used to sort the result-set in descending order. We consider each team separately by group by user location.

In [29]:
sqlc.sql("""
    SELECT user.location, COUNT(user.location) AS Time_Mention_Red_Sox 
    FROM tweets
    WHERE text LIKE '%Red Sox%'
    GROUP BY user.location
    ORDER BY COUNT(user.location) DESC

""").show(10)

+------------------+--------------------+
|          location|Time_Mention_Red_Sox|
+------------------+--------------------+
|        Boston, MA|                 332|
|     United States|                 185|
|   Los Angeles, CA|                 131|
|Massachusetts, USA|                 115|
| Dhaka, Bangladesh|                 113|
|        Bangladesh|                  96|
|            Boston|                  95|
|       Chicago, IL|                  80|
|      New York, NY|                  73|
|   California, USA|                  68|
+------------------+--------------------+
only showing top 10 rows



In [30]:
sqlc.sql("""
    SELECT user.location, COUNT(user.location) AS Time_Mention_Dodgers 
    FROM tweets
    WHERE text LIKE '%Dodgers%'
    GROUP BY user.location
    ORDER BY COUNT(user.location) DESC

""").show(10)

+-----------------+--------------------+
|         location|Time_Mention_Dodgers|
+-----------------+--------------------+
|  Los Angeles, CA|                 632|
|  California, USA|                 303|
|    United States|                 241|
|      Los Angeles|                 224|
|       Boston, MA|                 189|
|Dhaka, Bangladesh|                 113|
|           México|                 105|
|      Houston, TX|                 102|
|       Bangladesh|                  96|
|       California|                  93|
+-----------------+--------------------+
only showing top 10 rows



As the above two queries indicates, the locations where mentioned Dodgers and Red Sox the most is significantly different. It is not surprising that Boston, Ma mentioned Red Sox the most because it is a team from Boston and Los Angeles, CA for Dodgers as it is a Los Angeles team. So people tend to discuss their local teams.

### Q3.2 - Which Twitter users are being replied to the most?

Discuss your findings.

Hint: use the top-level `in_reply_to_screen_name` for this.

**Answer**

In [31]:
tweets.columns

['contributors',
 'coordinates',
 'created_at',
 'display_text_range',
 'entities',
 'extended_entities',
 'extended_tweet',
 'favorite_count',
 'favorited',
 'filter_level',
 'geo',
 'id',
 'id_str',
 'in_reply_to_screen_name',
 'in_reply_to_status_id',
 'in_reply_to_status_id_str',
 'in_reply_to_user_id',
 'in_reply_to_user_id_str',
 'is_quote_status',
 'lang',
 'place',
 'possibly_sensitive',
 'quote_count',
 'quoted_status',
 'quoted_status_id',
 'quoted_status_id_str',
 'quoted_status_permalink',
 'reply_count',
 'retweet_count',
 'retweeted',
 'retweeted_status',
 'source',
 'text',
 'timestamp_ms',
 'truncated',
 'user']

We count the frequency of "in reply to screen name " and show in a decending order toget the top 10 most replied twitter users. The result shows that RedSox is replied the most, as it shows 1360 times.

In [32]:
sqlc.sql("""
    SELECT in_reply_to_screen_name, COUNT(in_reply_to_screen_name)
    FROM tweets
    GROUP BY in_reply_to_screen_name
    ORDER BY COUNT(in_reply_to_screen_name) DESC
""").show(10)

+-----------------------+------------------------------+
|in_reply_to_screen_name|count(in_reply_to_screen_name)|
+-----------------------+------------------------------+
|                 RedSox|                          1360|
|                Dodgers|                           814|
|                    MLB|                           163|
|          DodgersNation|                           101|
|         DonnieWahlberg|                            77|
|           DAVIDprice24|                            75|
|                Rangers|                            59|
|             EJerezESPN|                            52|
|        MLBStatoftheDay|                            43|
|             VeniceMase|                            42|
+-----------------------+------------------------------+
only showing top 10 rows



### Q3.3 - Which 10 verified users have the most followers?  Which 10 unverified users have the most followers?

Provide both the screen names and follower counts for each.

Discuss your findings.

**Answer**

The MAX() function returns the largest value of the followers column. By switching verified column between "True" and "False", we can get verified and unverified users. We grouped by the screen name with a decending order.

In [33]:
sqlc.sql("""
    SELECT user.screen_name, MAX(user.followers_count) AS count
    FROM tweets 
    WHERE user.verified = 'true'
    GROUP BY user.screen_name
    ORDER BY count DESC

""").show(10)

+---------------+-------+
|    screen_name|  count|
+---------------+-------+
|            MLB|8296241|
|    lopezdoriga|7678758|
|El_Universal_Mx|4836985|
|   MagicJohnson|4685432|
|        Milenio|4165323|
|    MarketWatch|3611461|
| TheRealStanLee|3340350|
|       Newsweek|3324621|
|  NateSilver538|3156640|
|          Migos|2556415|
+---------------+-------+
only showing top 10 rows



In [34]:
sqlc.sql("""
    SELECT user.screen_name, MAX(user.followers_count) AS count
    FROM tweets 
    WHERE user.verified = 'false'
    GROUP BY user.screen_name
    ORDER BY count DESC

""").show(10)

+---------------+-------+
|    screen_name|  count|
+---------------+-------+
|   DRJAMESCABOT|2185433|
|       PAMsLOvE| 688117|
| mlbtraderumors| 666915|
| Miguel_Gurwitz| 568885|
|        ruleiro| 520622|
| RealKentMurphy| 443846|
|thebrittanyxoxo| 325224|
|FakeSportsCentr| 318999|
|    CelticsLife| 296296|
|    milenagimon| 226625|
+---------------+-------+
only showing top 10 rows



### Q3.4 - What are the most popular sets of hashtags among users with many followers?  Are they the same as among users with few followers?

Decide for yourself exactly how many followers you believe to be "many", and explain your decision.  You may use queries and statistics to support this decision if you wish.

Hint: if your sample tweet above does not include hashtags under the `entities` field, generate a new example by running the `shuf` command again until you find one that does.

Hint 2: the hashtag texts will be in an array, so you may need some functions you haven't used before.  If you're using SQL, see the docs for [Hive SQL](https://docs.treasuredata.com/articles/hive-functions) for details, (and consider `CONCAT_WS`, for example).

Discuss your findings.

**Answer**

We decide to use the average of followers_count(4105) as the benchmark.The CONCAT_WS() function adds two or more expressions together with a separator. We count the 10 most popular hashtags by ordering the count of entities hashtags in a decending order.

In [35]:
sqlc.sql("""
    SELECT COUNT(user) 
    FROM tweets
""").show(20)

+-----------+
|count(user)|
+-----------+
|     100000|
+-----------+



In [36]:
sqlc.sql("""
    SELECT AVG(user.followers_count)
    FROM tweets 
    
""").show()

+----------------------------------------------+
|avg(user.followers_count AS `followers_count`)|
+----------------------------------------------+
|                                    4104.94204|
+----------------------------------------------+



In [45]:
sqlc.sql("""
    SELECT concat_ws(',', entities.hashtags.text), COUNT(entities.hashtags.text)
    FROM tweets
    WHERE user.followers_count > 4105
    GROUP BY entities.hashtags.text
    ORDER BY COUNT(entities.hashtags.text) DESC
""").show(10)

+----------------------------------------------+---------------------------------------+
|concat_ws(,, entities.hashtags.text AS `text`)|count(entities.hashtags.text AS `text`)|
+----------------------------------------------+---------------------------------------+
|                                              |                                   3185|
|                                        RedSox|                                    331|
|                                   WorldSeries|                                    232|
|                            RedSox,WorldSeries|                                    143|
|                                       Dodgers|                                    116|
|                                    DAMAGEDONE|                                     56|
|                            WorldSeries,RedSox|                                     52|
|                                      DoDamage|                                     45|
|                    

### Q3.5 - Analyze common words in tweet text

Following the example in class, use `tweets.rdd` to find the most common interesting words in tweet text.  To keep it "interesting", add a filter that removes at least 10 common stop words found in tweets, like "a", "an", "the", and "RT" (you might want to derive these stop words from initial results).  To split lines into words, a simple split on text whitespace like we had in class is sufficient; you do not have to account for punctuation.

After you find the most common words, use dataframe or SQL queries to find patterns among how those words are used.  For example, are they more frequently used by Dodgers or Red Sox fans, or by people in one part of the country over another?  Explore and see what you can find, and discuss your findings.

Hint: don't forget all the word count pipeline steps we used earlier in class.

**Answer**

In [38]:
sw = ['a','in','of','to','','about','I','this', 'above','after','again','against','and','all','an','is','any','are','as','at','be','because','but','an','the','THE','did','do','am','before', 'being','RT','below','between','both','by','could','does','down','during','each','few','for','from','further','had','you','your','with','why','whom']

In [39]:
tweets.rdd.flatMap(lambda r: r['text'].split(' ')) \
       .filter(lambda w: w not in sw)\
       .map(lambda t: (t,1))\
       .reduceByKey(lambda a, b: a+b)\
       .takeOrdered(10, key = lambda pair: -pair[1])

[('World', 16829),
 ('Red', 14855),
 ('@RedSox:', 14766),
 ('Sox', 13374),
 ('Series', 12775),
 ('#WorldSeries', 11012),
 ('Dodgers', 9898),
 ('#RedSox', 9667),
 ('@RedSox', 8174),
 ('on', 6192)]

We define Dodgers or Red Sox fans as the people who has Dodgers or Redsox in their hashtags.     
Then, we can explore how the Dodgers or Red Sox fans mentions the words 'Worlds'. First, we count the tweets with each hashtagds and then count the text with the hashtags and mentioned the word.

In [40]:
sqlc.sql("""
    SELECT concat_ws(',', entities.hashtags.text) AS hashtags, COUNT(entities.hashtags.text)
    FROM tweets
    WHERE concat_ws(',', entities.hashtags.text) = 'RedSox'
    GROUP BY entities.hashtags.text
    ORDER BY COUNT(entities.hashtags.text) DESC
""").show(10)

+--------+---------------------------------------+
|hashtags|count(entities.hashtags.text AS `text`)|
+--------+---------------------------------------+
|  RedSox|                                   3583|
+--------+---------------------------------------+



In [41]:
sqlc.sql("""
    SELECT concat_ws(',', entities.hashtags.text) AS hashtags, COUNT(entities.hashtags.text)
    FROM tweets
    WHERE concat_ws(',', entities.hashtags.text) = 'RedSox'
    AND text LIKE '%World%'
    GROUP BY entities.hashtags.text
    ORDER BY COUNT(entities.hashtags.text) DESC
""").show(10)

+--------+---------------------------------------+
|hashtags|count(entities.hashtags.text AS `text`)|
+--------+---------------------------------------+
|  RedSox|                                    217|
+--------+---------------------------------------+



For Red Sox fans, 217 out 2583 (8.55%) tweets with Redsox hashtags mentioned the word 'World'

In [42]:
sqlc.sql("""
    SELECT concat_ws(',', entities.hashtags.text) AS hashtags, COUNT(entities.hashtags.text)
    FROM tweets
    WHERE concat_ws(',', entities.hashtags.text) = 'Dodgers'
    GROUP BY entities.hashtags.text
    ORDER BY COUNT(entities.hashtags.text) DESC
""").show(10)

+--------+---------------------------------------+
|hashtags|count(entities.hashtags.text AS `text`)|
+--------+---------------------------------------+
| Dodgers|                                   1456|
+--------+---------------------------------------+



In [43]:
sqlc.sql("""
    SELECT concat_ws(',', entities.hashtags.text) AS hashtags, COUNT(entities.hashtags.text)
    FROM tweets
    WHERE concat_ws(',', entities.hashtags.text) = 'Dodgers'
    AND text LIKE '%World%'
    GROUP BY entities.hashtags.text
    ORDER BY COUNT(entities.hashtags.text) DESC
""").show(10)

+--------+---------------------------------------+
|hashtags|count(entities.hashtags.text AS `text`)|
+--------+---------------------------------------+
| Dodgers|                                    113|
+--------+---------------------------------------+



For Dodgers fans, 113 out 1456 (7.76%) tweets with Dodgers hashtags mentioned the word 'World'.      
The difference is not significant,  Red Sox fans mentioned the word 'Worlds' slightly more than the Dodgers fans.

In [46]:
sqlc.sql("""
    SELECT user.location, COUNT(user.location) AS Time_Mention_Dodgers 
    FROM tweets
    WHERE text LIKE '%WorldSeries%'
    GROUP BY user.location
    ORDER BY COUNT(user.location) DESC

""").show(10)

+------------------+--------------------+
|          location|Time_Mention_Dodgers|
+------------------+--------------------+
|        Boston, MA|                 421|
|     United States|                 168|
|Massachusetts, USA|                 147|
|   Los Angeles, CA|                 141|
|            Boston|                 125|
|   California, USA|                  75|
|       Chicago, IL|                  68|
|     Massachusetts|                  63|
|        Texas, USA|                  59|
|       Los Angeles|                  55|
+------------------+--------------------+
only showing top 10 rows



Then we would love to explore which locations mention the word 'WorldSeries' the most. It is not surprising the first one is Boston, MA becuase Boston, MA as the most Red Sox fans, who mentions WorldSeries a lot.